# Importing the data set

In [6]:
import os
import pandas as pd

test_path = ".\data\\test\\"
pos_path = ".\data\\train\\pos\\"
neg_path = ".\data\\train\\neg\\"

neg_review_list = []
pos_review_list = []
review_list = []
test_set = []

for file in os.listdir(neg_path):
    file_path = os.path.join(neg_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {}
    entry['class'] = 0
    entry['raw_txt'] = fh.read()
    neg_review_list.append(entry)
    fh.close()
    
for file in os.listdir(pos_path):
    file_path = os.path.join(pos_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {}
    entry['class'] = 1
    entry['raw_txt'] = fh.read()
    pos_review_list.append(entry)
    fh.close()

review_list = pd.DataFrame(neg_review_list[:20] + pos_review_list[:20])
review_list_test = pd.DataFrame(neg_review_list[20:30] + pos_review_list[20:30])
pos_review_list = pd.DataFrame(pos_review_list)
neg_review_list = pd.DataFrame(neg_review_list)


print("h")

    class                                            raw_txt
0       0  Story of a man who has unnatural feelings for ...
1       0  Airport '77 starts as a brand new luxury 747 p...
2       0  This film lacked something I couldn't put my f...
3       0  Sorry everyone,,, I know this is supposed to b...
4       0  When I was little my parents took me along to ...
5       0  "It appears that many critics find the idea of...
6       0  The second attempt by a New York intellectual ...
7       0  I don't know who to blame, the timid writers o...
8       0  This film is mediocre at best. Angie Harmon is...
9       0  The film is bad. There is no other way to say ...
10      0  This film is one giant pant load. Paul Schrade...
11      0  The plot for Descent, if it actually can be ca...
12      0  Plot is not worth discussion even if it hints ...
13      0  This film is about a male escort getting invol...
14      0  This movie must be in line for the most boring...
15      0  A worn-out pl

Preprocessing


In [7]:
import re
from nltk.corpus import stopwords
import numpy as np
stopWords = set(stopwords.words('english'))

def processing(df):
    #lowering and removing punctuation
    df['processed'] = df['raw_txt'].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))
    
    #numerical feature engineering
    #total length of sentence
    df['length'] = df['processed'].apply(lambda x: len(x))
    #get number of words
    df['words'] = df['processed'].apply(lambda x: len(x.split(' ')))
    df['words_not_stopword'] = df['processed'].apply(lambda x: len([t for t in x.split(' ') if t not in stopWords]))
    #get the average word length
    df['avg_word_length'] = df['processed'].apply(lambda x: np.mean([len(t) for t in x.split(' ') if t not in stopWords]) if len([len(t) for t in x.split(' ') if t not in stopWords]) > 0 else 0)
    #get the average word length
    df['commas'] = df['raw_txt'].apply(lambda x: x.count(','))

    return df

df = processing(review_list)
df.head()
print(df)

    class                                            raw_txt  \
0       0  Story of a man who has unnatural feelings for ...   
1       0  Airport '77 starts as a brand new luxury 747 p...   
2       0  This film lacked something I couldn't put my f...   
3       0  Sorry everyone,,, I know this is supposed to b...   
4       0  When I was little my parents took me along to ...   
5       0  "It appears that many critics find the idea of...   
6       0  The second attempt by a New York intellectual ...   
7       0  I don't know who to blame, the timid writers o...   
8       0  This film is mediocre at best. Angie Harmon is...   
9       0  The film is bad. There is no other way to say ...   
10      0  This film is one giant pant load. Paul Schrade...   
11      0  The plot for Descent, if it actually can be ca...   
12      0  Plot is not worth discussion even if it hints ...   
13      0  This film is about a male escort getting invol...   
14      0  This movie must be in line fo

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator,TransformerMixin):
    def __init__(self, key):
        
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

text = Pipeline([
                ('selector', TextSelector(key='processed')),
                ('tfidf', TfidfVectorizer( stop_words='english'))
            ])

,class,raw_txt,processed,length,words,words_not_stopword,avg_word_length,commas
0,0,Story of a man who has unnatural feelings for ...,story of a man who has unnatural feelings for ...,644,112,63,6.365079,1
1,0,Airport '77 starts as a brand new luxury 747 p...,airport 77 starts as a brand new luxury 747 pl...,4324,801,484,5.456612,16
2,0,This film lacked something I couldn't put my f...,this film lacked something i couldnt put my fi...,776,141,64,6.375000,4
3,0,"Sorry everyone,,, I know this is supposed to b...",sorry everyone i know this is supposed to be a...,832,154,79,5.784810,14
4,0,When I was little my parents took me along to ...,when i was little my parents took me along to ...,2265,395,208,6.596154,26
5,0,"""It appears that many critics find the idea of...",it appears that many critics find the idea of ...,607,97,54,7.166667,3
6,0,The second attempt by a New York intellectual ...,the second attempt by a new york intellectual ...,1778,323,192,5.682292,14
7,0,"I don't know who to blame, the timid writers o...",i dont know who to blame the timid writers or ...,832,155,76,5.815789,8
8,0,This film is mediocre at best. Angie Harmon is...,this film is mediocre at best angie harmon is ...,363,66,35,6.142857,0
9,0,The film is bad. There is no other way to say ...,the film is bad there is no other way to say i...,1423,283,145,5.220690,3
